# Lab Assigment 2 Estimation of Apparent Motion

<p style="text-align:left;">
    José Pedro Cruz
    <span style="float:right;">
        up201504646
    </span>
</p>
<p style="text-align:left;">
    Martinho Figueiredo
    <span style="float:right;">
        up201506179
    </span>
</p>
<p style="text-align:left;">
    Nuno Nascimento
    <span style="float:right;">
        up201907933
    </span>
</p>


[![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)](https://github.com/martinhofigueiredo/VC)



In [1]:
import cv2
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import struct
%matplotlib widget


In [2]:
# Loads a yaml file with the parameters to run the code
def load_config(filename):
    with open(filename, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

In [3]:
# Find all jpgs and pngs in a folder and returns a list of their respective path 
def get_frame_paths(folder_path):
    frame_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            frame_path = os.path.join(folder_path, filename)
            frame_paths.append(frame_path)
    frame_paths.sort()
    return frame_paths

In [4]:
# Checks if the inut file is a directory and if it is it will try and get the frames inside the folder
# it will check if it is and mp4 and if it is it will read it and create a list of frames
def get_input_frames(input_path):
    if os.path.isdir(input_path):
        frame_paths = get_frame_paths(input_path)
        print(f"{frame_paths}")
        frames = [cv2.imread(frame_path) for frame_path in frame_paths]
    else :
        if input_path.endswith('.mp4'):
            cap = cv2.VideoCapture(input_path)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()
    return frames

In [2]:
def write_flow(filename, flow):
    height, width = flow.shape[:2]
    with open(filename, 'wb') as f:
        f.write(struct.pack('f', width))
        f.write(struct.pack('f', height))
        f.write(flow.astype(np.float32).tobytes())
  

In [3]:
def read_flow(filename):
    with open(filename, 'rb') as f:
        width = struct.unpack('f', f.read(4))[0]
        height = struct.unpack('f', f.read(4))[0]
        flow_bytes = f.read()

    flow = np.frombuffer(flow_bytes, np.float32)
    flow = flow.reshape(int(height), int(width), 2)

    return flow


def compute_angular_error(flow_gt, flow_est):
    angles_gt = np.arctan2(flow_gt[:, :, 1], flow_gt[:, :, 0])
    angles_est = np.arctan2(flow_est[:, :, 1], flow_est[:, :, 0])
    error = (np.sum((angles_gt - angles_est) ** 2, axis=2))
    return np.mean(error)



def compute_endpoint_error(flow_gt, flow_est):
    error = np.sqrt(np.sum((flow_gt - flow_est) ** 2, axis=2))
    return np.mean(error)

def compare_flow_files(file_gt, file_est):
    flow_gt = read_flow(file_gt)
    flow_est = read_flow(file_est)

    error_endpoint = compute_endpoint_error(flow_gt, flow_est)
    error_angular = compute_angular_error(flow_gt, flow_est)

    return error_endpoint, error_angular

# Lucas-Kanade

In [ ]:
print(f"Loading File \'config_LK.yml\'")

config = load_config('config_LK.yml')

input_path = os.getcwd()+config['input_path']

frames = get_input_frames(input_path)


feature_params = dict(
    maxCorners=config['max_corners'],
    qualityLevel=config['quality_level'],
    minDistance=config['min_distance'],
    blockSize=config['block_size']
)
lk_params = dict(
    winSize=(config['window_size'], config['window_size']),
    maxLevel=config['max_level'],
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, config['max_iterations'], config['epsilon'])
)

prev_frame = frames[0]
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
prev_roi = cv2.boundingRect(prev_corners)

flow_list = []  #List to store optical flow vectors

mask = np.zeros_like(prev_frame)

fig, ax = plt.subplots(figsize=(10, 7))

for curr_frame in frames[1:]:
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    curr_corners, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)

    good_prev_corners = prev_corners[status == 1]
    good_curr_corners = curr_corners[status == 1]

    flow = good_curr_corners - good_prev_corners # Calculate optical flow vectores
    
    flow_list.append(flow)

    M, _ = cv2.findHomography(good_prev_corners, good_curr_corners, cv2.RANSAC, config['ransac_threshold'])

    warped_roi = cv2.warpPerspective(prev_frame, M, (curr_frame.shape[1], curr_frame.shape[0]))
    warped_mask = cv2.warpPerspective(mask, M, (curr_frame.shape[1], curr_frame.shape[0]))

    output = curr_frame.copy()
    output[warped_mask != 0] = 0
    output = cv2.add(output, warped_roi)

    ax.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    ax.axis('off')
    plt.tight_layout()
    clear_output(wait=True)
    display(fig)

    prev_gray = curr_gray.copy()
    prev_corners = good_curr_corners.reshape(-1, 1, 2)
    prev_roi = cv2.boundingRect(prev_corners)
flow_array = np.array(flow_list)

output_file = 'Optical_flow.flo'
write_flow(output_file,flow_array)
cv2.destroyAllWindows()


# Horn-Shunchk

# Bench Marking

## Import Dataset

In [1]:
# IMport dataset with public ground truth

import os
import subprocess
import wget
import zipfile

# Specify the URLs of the files to download
url1 = "https://vision.middlebury.edu/flow/data/comp/zip/other-gt-flow.zip"
url2 = "https://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip"

# Specify the destination folder to store the downloaded files
destination_folder = "dataset/"

# Download the files using wget
file1 = wget.download(url1, out=destination_folder)
file2 = wget.download(url2, out=destination_folder)

# Unzip the files
with zipfile.ZipFile(file1, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

with zipfile.ZipFile(file2, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

# Remove the zip files if needed
os.remove(file1)
os.remove(file2)